In [ ]:
import glob
import itertools
import signal
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import skimage
from cytoolz import compose

# import cv2
from matplotlib.offsetbox import AnnotationBbox, DrawingArea, OffsetImage, TextArea
from skimage.color import rgb2gray
from skimage.feature import blob_dog, blob_doh, blob_log

from paulssonlab.projects.molecule_counting.matriarch_stub import permute_labels
from paulssonlab.projects.molecule_counting.segmentation import invert, segment

mpl.rcParams["figure.figsize"] = (5, 5)

In [ ]:
# To check:
#     single point vs gradual gradient (i.e. size of the blob)
#     gradient of the intensity of the blob (i.e. one clear blob and no background or more diffuse blob)
#     different cell shape
#     position of blob within cell
#     intensity of the blob vs background of cell
# Test on 2d Gaussian and hat function

In [ ]:
def generate_test_image(
    size=(200, 200),
    mask=None,
    shapes=[["circle", [100, 100], 10, 100]],
    savename="test",
    background=None,
    cell_background=None,
):
    # shape = ['rings'=concentric circles, 'circle'=circle, 'gaussian'=2d gaussian]
    # shape == 'circle': ['circle',centre position indices,radius,intensity]
    # shape == 'rings': ['rings',centre position indices,
    # radius of smallest circle, intensity, width of rings, distance between rings]
    # shape == 'gaussian': ['gaussian', centre position indices, stdevs, intensity] (mean = position)
    # mask should be False where the cell is.
    if not background:
        image = np.zeros(size)
    elif background[0] == "gaussian":
        mean, stdev = background[1:]
        image = np.random.normal(loc=mean, scale=stdev, size=size)
    if np.size(mask) > 1:
        image = np.ma.array(image, mask=mask)
        if cell_background:
            if cell_background[0] == "gaussian":
                mean, stdev = cell_background[1:]
                cell_noise = np.random.normal(loc=mean, scale=stdev, size=size)
                image += cell_noise
    for shape in shapes:
        if shape[0] == "circle" or shape[0] == "rings":
            image_to_add = np.zeros(size)
            position, r, intensity = shape[1:]
            ranges = np.arange(np.floor(-r), np.ceil(r) + 1)
            rel_points = np.where(
                (ranges[np.newaxis, :]) ** 2 + (ranges[:, np.newaxis]) ** 2 < r**2
            )
            points = [
                rel_points[0] + position[0] - np.ceil(r),
                rel_points[1] + position[1] - np.ceil(r),
            ]
            image_to_add[
                list(map(int, points[0])), list(map(int, points[1]))
            ] = intensity
        #         if shape[0] == 'rings':

        elif shape[0] == "gaussian":
            position, stdevs, intensity = shape[1:]
            vals0, vals1 = np.meshgrid(np.arange(size[0]), np.arange(size[1]))
            image_to_add = np.exp(
                -(
                    (vals0 - position[0]) ** 2 / (2 * stdevs[0]) ** 2
                    + (vals1 - position[1]) ** 2 / (2 * stdevs[1]) ** 2
                )
            )
            image_to_add *= intensity
        image += image_to_add
    if savename:
        np.savetxt(savename, image)
    return image

In [ ]:
# get mask (need to invert current mask):
mask = np.genfromtxt("cellmask")
mask = np.ma.masked_array(mask, np.logical_not(mask)).mask

In [ ]:
# test_image = generate_test_image(size=(150,150),shapes=[['gaussian',[70,70],[4,4],100]]
#                                  ,savename='test_image')

# test_image = generate_test_image(size=(150,150),shapes=[['circle',[70,70],1,100]],
#                                  savename='test_image',mask=mask,background=['gaussian',10,3],
#                                  cell_background = ['gaussian',20,3])

# test_image = generate_test_image(size=(150,150),shapes=[['circle',[70,70],5,100],['circle',[70,50],5,100],
#                                                        ['circle',[70,90],5,100]],
#                                  savename='test_image')

# test_image = generate_test_image(size=(150,150),shapes=[['circle',[70,70],5,10],['circle',[70,50],2,200],
#                                                        ['circle',[80,90],5,100],['circle',[64,90],8,80],
#                                                        ['circle',[80,100],8,30]],
#                                  savename='test_image')

# test_image = generate_test_image(size=(150,150),shapes=[['gaussian',[70,70],[5,5],100]]
#                                  ,savename='test_image',mask=mask)

test_image = generate_test_image(
    size=(150, 150),
    shapes=[["gaussian", [70, 70], [1, 1], 100]],
    background=["gaussian", 10, 3],
    savename="test_image",
    mask=mask,
    cell_background=["gaussian", 20, 3],
)

# test_image = generate_test_image(size=(150,150),shapes=[['gaussian',[38,70],[3,3],0]],
#                                  savename='test_image',mask=mask,background=['gaussian',10,3],
#                                  cell_background = ['gaussian',20,3])

# test_image = generate_test_image(size=(150,150),shapes=[['gaussian',[38,70],[2,2],100],
#                                                        ['gaussian',[100,80],[2,2],100]],
#                                  savename='test_image',mask=mask,background=['gaussian',10,3],
#                                  cell_background = ['gaussian',20,3])

# test_image = generate_test_image(size=(150,150),shapes=[['gaussian',[38,70],[2,2],40],
#                                                        ['gaussian',[100,80],[4,4],35],
#                                                        ['gaussian',[70,80],[4,2],40],
#                                                        ['gaussian',[85,65],[5,2],20],
#                                                        ['gaussian',[40,68],[2,6],10],
#                                                        ['gaussian',[76,75],[2,2],20],
#                                                        ['gaussian',[50,72],[3,2],30],
#                                                        ['gaussian',[55,79],[4,2],20],
#                                                        ['gaussian',[45,63],[3,5],10]],
#                                  savename='test_image',mask=mask,background=['gaussian',10,3],
#                                  cell_background = ['gaussian',20,3])

plt.imshow(test_image.data)

## 